In [1]:
import pickle 
import requests 
import json 
import pandas as pd 
import numpy as np 
import os 
import time
import subprocess 
import parse_xml
import xml.etree.ElementTree as ET 
import urllib.request 
import lxml.etree 
import lxml.html 
import lxml
from parse_xml import *
from lxml import etree
from urllib.error import HTTPError
from lxml import etree
from bs4 import BeautifulSoup
from lxml import html as lxmlhtml
from multiprocessing import Pool importcpu_count importProcess
from functools import partial
from biomedkg_utils import *
import biomed_apis
from biomed_apis import *
direct = 'output/compound2compound/'

## Source: MRCONSO.RRF
- Edges:
    - MeSH <-is-> UMLS
    - DrugBank <-is-> UMLS
- Files:
    - MRCONSO.RRF (# Manually download from here and upload to the server (requires login) https://www.nlm.nih.gov/research/umls/licensedcontent/umlsknowledgesources.html)

In [3]:
#!cp '../../DATA/UMLS/Metathesaurus/umls-2022AA-mrconso.zip' input/
#!unzip input/umls-2022AA-mrconso.zip
#!mv MRCONSO.RRF input/MRCONSO.RRF

## UMLS -is- MeSH, DrugBank

In [5]:
'''Ontologies in MRCONSO.RRF'''
mrconso_ontologies = set()

with open('input/MRCONSO.RRF') as fin:
    for line in fin:
        line = line.strip().split('|')
        ontology = line[11]
        mrconso_ontologies.add(ontology)

mrconso_ontologies

{'AIR',
 'ALT',
 'AOD',
 'AOT',
 'ATC',
 'BI',
 'CCC',
 'CCPSS',
 'CCS',
 'CCSR_ICD10CM',
 'CCSR_ICD10PCS',
 'CDCREC',
 'CDT',
 'CHV',
 'COSTAR',
 'CPM',
 'CPT',
 'CPTSP',
 'CSP',
 'CST',
 'CVX',
 'DDB',
 'DMDICD10',
 'DMDUMD',
 'DRUGBANK',
 'DSM-5',
 'DXP',
 'FMA',
 'GO',
 'GS',
 'HCDT',
 'HCPCS',
 'HCPT',
 'HGNC',
 'HL7V2.5',
 'HL7V3.0',
 'HPO',
 'ICD10',
 'ICD10AE',
 'ICD10AM',
 'ICD10AMAE',
 'ICD10CM',
 'ICD10DUT',
 'ICD10PCS',
 'ICD9CM',
 'ICF',
 'ICF-CY',
 'ICNP',
 'ICPC',
 'ICPC2EDUT',
 'ICPC2EENG',
 'ICPC2ICD10DUT',
 'ICPC2ICD10ENG',
 'ICPC2P',
 'ICPCBAQ',
 'ICPCDAN',
 'ICPCDUT',
 'ICPCFIN',
 'ICPCFRE',
 'ICPCGER',
 'ICPCHEB',
 'ICPCHUN',
 'ICPCITA',
 'ICPCNOR',
 'ICPCPOR',
 'ICPCSPA',
 'ICPCSWE',
 'JABL',
 'KCD5',
 'LCH',
 'LCH_NW',
 'LNC',
 'LNC-DE-AT',
 'LNC-DE-DE',
 'LNC-EL-GR',
 'LNC-ES-AR',
 'LNC-ES-ES',
 'LNC-ES-MX',
 'LNC-ET-EE',
 'LNC-FR-BE',
 'LNC-FR-CA',
 'LNC-FR-FR',
 'LNC-IT-IT',
 'LNC-KO-KR',
 'LNC-NL-NL',
 'LNC-PL-PL',
 'LNC-PT-BR',
 'LNC-RU-RU',
 'LNC-TR-TR',
 '

In [6]:
direct = 'output/compound2compound/'
umls2mesh, mesh2umls = dict(), dict()
umls2db, db2umls = dict(), dict()

with open('input/MRCONSO.RRF') as fin:
    for line in fin:
        line = line.strip().split('|')
        umls = line[0]
        ontology = line[11]
        
        if 'MSH' == ontology:
            mesh = line[10]
            umls2mesh.setdefault(umls,set()).add(mesh)
            mesh2umls.setdefault(mesh,set()).add(umls)
            
        if 'DRUGBANK' in line:    
            db_ind = (line.index('DRUGBANK'))-2
            db = line[db_ind]
            db2umls.setdefault(db,set()).add(umls)    
            umls2db.setdefault(umls,set()).add(db)

    
'''Export'''
db2umls_json = switch_dictset_to_dictlist(db2umls)
umls2db_json = switch_dictset_to_dictlist(umls2db)
umls2mesh_json = switch_dictset_to_dictlist(umls2mesh)
mesh2umls_json = switch_dictset_to_dictlist(mesh2umls)
json.dump(db2umls_json, open(direct+'db2umls.json','w'))
json.dump(umls2db_json, open(direct+'umls2db.json','w'))
json.dump(umls2mesh_json, open('output/otherMappings/umls2mesh.json','w'))
json.dump(mesh2umls_json, open('output/otherMappings/mesh2umls.json','w'))

print(len(umls2mesh))
print(len(mesh2umls))
print(len(db2umls))
print(len(umls2db))

455672
348733
8368
10118


In [ ]:
# switch mesh2umls_all to mesh2umls

In [ ]:

456219
348946
8368
10118

## Source: UMLS API
- Note: Does not appear to add more than MRCONSO.RRF, at least for these UMLS IDs provided from MRCONSO.RRF, the UMLS IDs with DrugBank ID alignments 

In [59]:
'''UMLS -is-> MeSH (API) multiprocessing'''
### API
#multiprocess_a_dict_of_lists_values(db2umls, mesh2umls_query)


### Merge temp files 
umls2meshAPI, mesh2umlsAPI = dict(), dict()

for b_id in range(procs):
    temp_path = direct+'temp'+str(b_id)+'.txt'
    with open(temp_path,'r') as fin:
        for i, line in enumerate(fin):
            print(i, end='\r')
            umls = line.split('|')[0].strip()
            mesh = line.split('|')[1].strip()

            umls2meshAPI.setdefault(umls,set()).add(mesh)
            mesh2umlsAPI.setdefault(mesh,set()).add(umls)
    #os.remove(temp_path)

In [60]:
umls2mesh_json = switch_dictset_to_dictlist(umls2mesh)
mesh2umls_json = switch_dictset_to_dictlist(mesh2umls)
json.dump(umls2mesh_json, open('output/otherMappings/umls2mesh.json','w'))
json.dump(mesh2umls_json, open('output/otherMappings/mesh2umls.json','w'))

print(len(umls2mesh))
print(len(mesh2umls))

NameError: name 'umls2mesh' is not defined

## Source: DrugBank
- Edges:
    - DrugBank -is- CAS, UNII, PubChem Substance, PubChem Compound, ChemSpider, Chebi, Chembl, KEGG Compound, KEGG Drug, UniProt

- Files:
    
    - https://go.drugbank.com/releases/5-1-9/downloads/all-full-database

In [ ]:
tree = ET.parse('input/full_drugbank.xml')
root = tree.getroot()

In [ ]:
db2cas, db2unii, atc2db, db2atc, db2onts = dict(), dict(), dict(), dict(), dict()
cas2db, unii2db = dict(), dict()      
db2mesh, mesh2db = dict(), dict()
dbs = set()


'''DrugBank -is _______'''
db2chembl, db2chebi, db2pubchemcomp, db2chemspider, db2pubchemsub = dict(), dict(), dict(), dict(), dict()
db2keggdrug, db2keggcompound, db2oldttd, db2uniprot, db2pharmgkb, db2wikipedia = dict(), dict(), dict(), dict(), dict(), dict()
chembl2db, chebi2db, pubchemcomp2db, chemspider2db, pubchemsub2db = dict(), dict(), dict(), dict(), dict()
keggdrug2db, keggcompound2db, oldttd2db, uniprot2db, pharmgkb2db, wikipedia2db = dict(), dict(), dict(), dict(), dict(), dict()


for ele in root:
    db = ParseXML.getID(ele)
    dbs.add(db)
    
    # DrugBank -is- CAS
    cas = ele.find('{http://www.drugbank.ca}cas-number').text
    if cas != None:
        db2cas[db] = [cas]                   # 1 DrugBank is 1 CAS
        cas2db.setdefault(cas,set()).add(db) # 1 CAS is 1+ DrugBank (usually 1)
        
    # DrugBank -is- UNII
    unii = ParseXML.getUNII(ele)
    if(unii != None):
        db2unii[db] = [unii]                   # 1 to 1
        unii2db.setdefault(unii,set()).add(db) # 1 to 1
    
    # DrugBank -is- ATC
    atcs = ParseXML.getATCCode(ele)
    if(atcs != []):
        db2atc[db] = atcs
        for atc in atcs:
            atc2db.setdefault(atc,set()).add(db)
        
    # DrugBank -is- Other ontologies
    ext_id_dicts = ParseXML.getExternalIDs(ele)
    db2onts[db] = ext_id_dicts
    
    
# Ontology to DrugBank Dictionary
allonts_d, allonts = dict(), set()
for drug, onts in db2onts.items():
    for ont in onts:
        allonts_d.setdefault(ont,set()).add(drug)   
        allonts.add(ont)

for db, onts in db2onts.items():
    # DrugBank -is- ChEMBL
    if 'ChEMBL' in onts:
        db2chembl[db] = onts['ChEMBL']
        for chembl in onts['ChEMBL']:
            chembl2db.setdefault(chembl,set()).add(db)
    
    # DrugBank -is- ChEBI
    if 'ChEBI' in onts:
        db2chebi[db] = onts['ChEBI']
        for chebi in onts['ChEBI']:
            chebi2db.setdefault(chebi,set()).add(db)

    # DrugBank -is- PubChem Compound
    if 'PubChem Compound' in onts:
        db2pubchemcomp[db] = onts['PubChem Compound']
        for pubchemcomp in onts['PubChem Compound']:
            pubchemcomp2db.setdefault(pubchemcomp,set()).add(db)

    # DrugBank -is- PubChem Substance
    if 'PubChem Substance' in onts:
        db2pubchemsub[db] = onts['PubChem Substance']
        for pubchemsub in onts['PubChem Substance']:
            pubchemsub2db.setdefault(pubchemsub,set()).add(db)
    
    
    # DrugBank -is- ChemSpider
    if 'ChemSpider' in onts:
        db2chemspider[db] = onts['ChemSpider']
        for chemspider in onts['ChemSpider']:
            chemspider2db.setdefault(chemspider,set()).add(db)

    # DrugBank -is- KEGG Drug
    if 'KEGG Drug' in onts:
        db2keggdrug[db] = onts['KEGG Drug']
        for keggdrug in onts['KEGG Drug']:
            keggdrug2db.setdefault(keggdrug,set()).add(db)

    # DrugBank -is- KEGG Compound
    if 'KEGG Compound' in onts:
        db2keggcompound[db] = onts['KEGG Compound']
        for keggcomp in onts['KEGG Compound']:
            keggcompound2db.setdefault(keggcomp,set()).add(db)
        
    # DrugBank -is- Therapeutic Targets Database
    if 'Therapeutic Targets Database' in onts:
        db2oldttd[db] = onts['Therapeutic Targets Database']
        for oldttd in onts['Therapeutic Targets Database']:
            oldttd2db.setdefault(oldttd,set()).add(db)
        
    # DrugBank -is- UniProt
    if 'UniProtKB' in onts:
        db2uniprot[db] = onts['UniProtKB']
        for uniprot in onts['UniProtKB']:
            uniprot2db.setdefault(uniprot,set()).add(db)
            
    # DrugBank -is- PharmGKB
    if 'PharmGKB' in onts:
        db2pharmgkb[db] = onts['PharmGKB']  
        for pharm in onts['PharmGKB']:
            pharmgkb2db.setdefault(pharm,set()).add(db)
        
    # DrugBank -is- Wikipedia
    if 'Wikipedia' in onts:
        db2wikipedia[db] = onts['Wikipedia']
        for wiki in onts['Wikipedia']:
            wikipedia2db.setdefault(wiki,set()).add(db)
    
    
'''Print numbers'''  
# Display the number of DrugBank-provided xref alignments
print('DrugBank-provided external mappings:')
for ont,drugs in allonts_d.items():
    print(len(drugs),'DrugBank -is-',ont)
print(len(db2cas), 'DrugBank -is- CAS')
print(len(cas2db), 'CAS -is/are- DrugBank')
print(len(db2unii), 'DrugBank -is- UNII')
print(len(db2atc), 'DrugBank -is/are- ATC')
print(len(atc2db), 'ATC -is/are- DrugBank')


'''Export alginment mappings'''
chembl2db = switch_dictset_to_dictlist(chembl2db)
chebi2db = switch_dictset_to_dictlist(chebi2db)
pubchemcomp2db = switch_dictset_to_dictlist(pubchemcomp2db)
chemspider2db = switch_dictset_to_dictlist(chemspider2db)
pubchemsub2db = switch_dictset_to_dictlist(pubchemsub2db)
keggdrug2db = switch_dictset_to_dictlist(keggdrug2db)
keggcompound2db = switch_dictset_to_dictlist(keggcompound2db)
wikipedia2db = switch_dictset_to_dictlist(wikipedia2db)
oldttd2db = switch_dictset_to_dictlist(oldttd2db)
uniprot2db = switch_dictset_to_dictlist(uniprot2db)
pharmgkb2db = switch_dictset_to_dictlist(pharmgkb2db)
unii2db = switch_dictset_to_dictlist(unii2db)
db2unii = switch_dictset_to_dictlist(db2unii)
atc2db = switch_dictset_to_dictlist(atc2db)
db2atc = switch_dictset_to_dictlist(db2atc)
cas2db = switch_dictset_to_dictlist(cas2db)
db2cas = switch_dictset_to_dictlist(db2cas)


dicts = [atc2db, db2atc, cas2db, db2cas, db2unii, unii2db, db2chembl,
         db2chebi, db2pubchemcomp, db2chemspider,
         db2pubchemsub, chembl2db, chebi2db, pubchemcomp2db,
         chemspider2db, pubchemsub2db, db2keggdrug,
         db2keggcompound, db2oldttd, db2uniprot, db2pharmgkb,
         db2wikipedia, keggdrug2db, keggcompound2db, oldttd2db,
         uniprot2db, pharmgkb2db, wikipedia2db]

locals_copy = locals().copy()

print('\nExporting...')
for i in range(len(dicts)):    
    dict_name = [k for k, v in locals_copy.items() if v == dicts[i]][0]
    json.dump(dicts[i], open(direct+''+dict_name+'.json','w'))
    print(dict_name)
    
# If the name of a variable doesn't seem right, delete the variable name (e.g., del __97)

In [ ]:
db_to_atc = json.load(open('output/compound2compound/db2atc.json'))

with open('output/compound2compound/drugbank_drug_to_atc_class.csv','w') as fout:
    writer = csv.writer(fout)
    writer.writerow(['Compound (DrugBank)','Compound (ATC)','-is->'])
    rel = '-is-'
    
    for drugbank, atcs in db_to_atc.items():
        for atc in atcs:
            writer.writerow([drugbank, atc, rel])

### DrugBank to UNII to MeSH

In [93]:
db2unii = json.load(open(direct+'db2unii.json'))
unii2db = json.load(open(direct+'unii2db.json'))

In [94]:
'''MeSH API/Scraper'''
#multiprocess_a_dict_of_lists_values(db2unii, unii2mesh_meshapi)

### Make unii2mesh: Merge the temp files into a dictionary
unii2mesh, mesh2unii = dict(), dict()
procs = cpu_count()

for b_id in range(procs):
    temp_path = direct+'temp_unii2mesh'+str(b_id)+'.txt'
    with open(temp_path) as fin:
        for line in fin:
            
            # UNII
            unii = line.split('|')[0].strip()
            # MeSH
            mesh = line.split('|')[1].strip()    
            # UNII -is- MeSH
            unii2mesh.setdefault(unii, set()).add(mesh)
            # MeSH -is- UNII
            mesh2unii.setdefault(mesh, set()).add(unii)

    #os.remove(temp_path)

print(len(unii2mesh))
print(len(mesh2unii))

2134
2111


In [95]:
'''MyChem.info API'''
#multiprocess_a_dict_of_lists_values(db2unii, unii2mesh_mycheminfoapi)

### Merge the temp unii2mesh files into a dictionary
procs = cpu_count()
for b_id in range(procs):
    temp_path = direct+'temp_unii2mesh_mycheminfo'+str(b_id)+'.txt'
    with open(temp_path) as fin:
        for line in fin:
            
            unii = line.split('|')[0].strip()
            mesh = line.split('|')[1].strip()            
            unii2mesh.setdefault(unii, set()).add(mesh)
            mesh2unii.setdefault(mesh, set()).add(unii)
            
    #os.remove(temp_path)

print(len(unii2mesh))
print(len(mesh2unii))

json.dump(switch_dictset_to_dictlist(unii2mesh), open(direct+'unii2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2unii), open(direct+'mesh2unii.json','w'))

3044
3022


## DrugBank is CAS

In [96]:
'''Inxight Drugs API/Scraper'''
mesh2cas, cas2mesh = dict(), dict()
           
# multiprocess_a_list(dbs, db2meshorcas_inxightAPI)

print(len(db2mesh))
print(len(db2cas))



'''Merge Files: DrugBank-is-CAS, DrugBank-is-MeSH'''
db2mesh, mesh2db = dict(), dict()
db2cas = json.load(open(direct+'db2cas.json'))
cas2db = json.load(open(direct+'cas2db.json'))
db2cas = switch_dictlist_to_dictset(db2cas)
cas2db = switch_dictlist_to_dictset(cas2db)

procs = cpu_count()
for b_id in range(procs):
    
    ### DrugBank-is-MeSH
    temp_db2mesh_path = direct+'temp_db2mesh_inxightapi'+str(b_id)+'.txt'
    with open(temp_db2mesh_path) as fin1:
        for line in fin1:            
            db = line.split('|')[0].strip()
            mesh = line.split('|')[1].strip() 
            db2mesh.setdefault(db, set()).add(mesh)
            mesh2db.setdefault(mesh, set()).add(db)
        #os.remove(temp_path)

    ### DrugBank-is-CAS
    temp_db2cas_path = direct+'db2cas_inxight'+str(b_id)+'.txt'
    with open(temp_db2cas_path) as fin2:
        for line in fin2:
            db = line.split('|')[0].strip()
            cases = line.split('|')[1:]
            for cas in cases:
                cas = cas.strip()
                cas2db.setdefault(cas, set()).add(db)
                db2cas.setdefault(db, set()).add(cas)
        #os.remove(temp_path)

# Export
json.dump(switch_dictset_to_dictlist(db2cas), open(direct+'db2cas.json','w'))
json.dump(switch_dictset_to_dictlist(cas2db), open(direct+'cas2db.json','w'))
json.dump(switch_dictset_to_dictlist(db2mesh), open(direct+'db2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2db), open(direct+'mesh2db.json','w'))
print(len(db2mesh))
print(len(db2cas))

8013
9182
1676
9182


In [97]:
'''MeSH API/Scraper: CAS-is-MeSH'''
#multiprocess_a_dict_of_lists_values(db2cas, cas2mesh_meshapi)

### Make CAS-is-MeSH: Merge the temp cas2mesh files into a dictionary
cas2mesh = switch_dictlist_to_dictset(cas2mesh)
mesh2cas = switch_dictlist_to_dictset(mesh2cas)

procs = cpu_count()
for b_id in range(procs):
    temp_path = direct+'temp_cas2mesh'+str(b_id)+'.txt'
    with open(temp_path) as fin:
        for line in fin:
            
            cas = line.split('|')[0].strip()
            mesh = line.split('|')[1].strip()            
            cas2mesh.setdefault(cas, set()).add(mesh)
            mesh2cas.setdefault(mesh, set()).add(cas)
            
    #os.remove(temp_path)
    
print(len(cas2mesh))
print(len(mesh2cas))

# Prepare to export
cas2mesh = switch_dictset_to_dictlist(cas2mesh)
mesh2cas = switch_dictset_to_dictlist(mesh2cas)

# Export
json.dump(cas2mesh, open(direct+'cas2mesh.json','w'))
json.dump(mesh2cas, open(direct+'mesh2cas.json','w'))

2371
2332


## DrugBank is MeSH

In [98]:
db2umls = json.load(open(direct+'db2umls.json'))
umls2mesh = json.load(open('output/otherMappings/umls2mesh.json'))
cas2db = json.load(open(direct+'cas2db.json'))
unii2db = json.load(open(direct+'unii2db.json'))
unii2mesh = json.load(open(direct+'unii2mesh.json'))
cas2mesh = json.load(open(direct+'cas2mesh.json'))

In [99]:
print('MeSH-is-DrugBank:',len(mesh2db))
print('Drugbank-is-MeSH:',len(db2mesh))

MeSH-is-DrugBank: 1668
Drugbank-is-MeSH: 1676


In [100]:
'''DrugBank-is-UMLS-is-MeSH'''
for db,umls_list in db2umls.items():
    for umls in umls_list:
        try:
            meshes = umls2mesh[umls]
            for mesh in meshes:
                db2mesh.setdefault(db,set()).add(mesh)
                mesh2db.setdefault(mesh,set()).add(db)
        except:
            continue
            
print('MeSH-is-DrugBank:',len(mesh2db))
print('Drugbank-is-MeSH:',len(db2mesh))

MeSH-is-DrugBank: 6278
Drugbank-is-MeSH: 6085


In [101]:
'''DrugBank-is-UNII-is-MeSH'''
for unii, meshes in unii2mesh.items():
    try:
        dbs = unii2db[unii]
        for db in dbs:
            for mesh in meshes:
                db2mesh.setdefault(db,set()).add(mesh)
                mesh2db.setdefault(mesh,set()).add(db)
    except:
        continue
              
print('MeSH-is-DrugBank:',len(mesh2db))
print('Drugbank-is-MeSH:',len(db2mesh))

MeSH-is-DrugBank: 6833
Drugbank-is-MeSH: 6627


In [102]:
'''DrugBank-is-CAS-is-MESH'''
for cas, meshes in cas2mesh.items():
    try:
        db = cas2db[cas][0]
        for mesh in meshes:
            db2mesh.setdefault(db,set()).add(mesh)
            mesh2db.setdefault(mesh,set()).add(db)
    except:
        continue
              
print('MeSH-is-DrugBank:',len(mesh2db))
print('Drugbank-is-MeSH:',len(db2mesh))

json.dump(switch_dictset_to_dictlist(db2mesh), open(direct+'db2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2db), open(direct+'mesh2db.json','w'))

MeSH-is-DrugBank: 7379
Drugbank-is-MeSH: 7172


## CAS is MeSH

In [105]:
# Import
cas2mesh = switch_dictlist_to_dictset(cas2mesh)
print('CAS-is-MeSH',len(cas2mesh), len(mesh2cas))

# Add items
key2mesh_via_db(cas2db, db2mesh, cas2mesh, mesh2cas)
print('CAS-is-MeSH',len(cas2mesh), len(mesh2cas))

# Export
json.dump(switch_dictset_to_dictlist(cas2mesh), open(direct+'cas2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2cas), open(direct+'mesh2cas.json','w'))

CAS-is-MeSH 2371 2332
CAS-is-MeSH 6992 7012


## ChEBI is MeSH

In [106]:
# Import
chebi2db = json.load(open(direct+'chebi2db.json'))

# Add items
chebi2mesh, mesh2chebi = dict(), dict()
print('ChEBI-is-MeSH', len(chebi2mesh))
key2mesh_via_db(chebi2db, db2mesh, chebi2mesh, mesh2chebi)
print('ChEBI-is-MeSH', len(chebi2mesh))

# Export
json.dump(switch_dictset_to_dictlist(chebi2mesh), open(direct+'chebi2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2chebi), open(direct+'mesh2chebi.json','w'))

ChEBI-is-MeSH 0
ChEBI-is-MeSH 4026


## InchiKey & SMILES -is- MeSH & DrugBank

In [107]:
# INSTRUCTIONS: Download from https://go.drugbank.com/releases/5-1-9/downloads/all-structure-links
# You need a DrugBank account to download this file
# ! unzip 'input/drugbank_all_structure_links.csv.zip'
# moved the file to input
structure_df = pd.read_csv('input/structure links.csv')

'''InchiKey & SMILES -is- DrugBank'''
db2inchikey, inchikey2db = dict(), dict()

db2smiles, smiles2db = dict(), dict()

for i in range(len(structure_df)):
    
    # DrugBank ID
    db = structure_df['DrugBank ID'].iloc[i]
    
    # Inchi Key
    inchikey = structure_df['InChIKey'].iloc[i]
    if type(inchikey) == str:
        db2inchikey.setdefault(db,set()).add(inchikey)
        inchikey2db.setdefault(inchikey,set()).add(db)
        
    # SMILES
    smiles = structure_df['SMILES'].iloc[i]
    if type(smiles) == str:
        db2smiles.setdefault(db,set()).add(smiles)
        smiles2db.setdefault(smiles, set()).add(db)
        
# Export
json.dump(switch_dictset_to_dictlist(db2inchikey), open(direct+'db2inchikey.json','w'))
json.dump(switch_dictset_to_dictlist(inchikey2db), open(direct+'inchikey2db.json','w'))
json.dump(switch_dictset_to_dictlist(db2smiles), open(direct+'db2smiles.json','w'))
json.dump(switch_dictset_to_dictlist(smiles2db), open(direct+'smiles2db.json','w'))


'''InchiKey & SMILES -is- MeSH'''
# InchiKey -is- MeSH
inchis = list(json.load(open(direct+'inchikey2db.json')).keys())
inchikey2mesh, mesh2inchikey = dict(), dict()
inchi2mesh_mycheminfoAPI(inchis, inchikey2mesh, mesh2inchikey)
key2mesh_via_db(inchikey2db, db2mesh, inchikey2mesh, mesh2inchikey)
print('InchiKey-is-MeSH', len(inchikey2mesh), len(mesh2inchikey))

# SMILES -is- MeSH
smiles2mesh, mesh2smiles = dict(), dict()
key2mesh_via_db(smiles2db, db2mesh, smiles2mesh, mesh2smiles)
print('SMILES-is-MeSH', len(smiles2mesh), len(mesh2smiles))

# Export
json.dump(switch_dictset_to_dictlist(smiles2mesh), open(direct+'smiles2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2smiles), open(direct+'mesh2smiles.json','w'))
json.dump(switch_dictset_to_dictlist(inchikey2mesh), open(direct+'inchikey2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2inchikey), open(direct+'mesh2inchikey.json','w'))

InchiKey-is-MeSH 6380 6588
SMILES-is-MeSH 5941 6115


In [ ]:
InchiKey-is-MeSH 6380 6534
SMILES-is-MeSH 5941 5936

## CAS is MeSH

In [108]:
# Import
cas2mesh = switch_dictlist_to_dictset(cas2mesh)
print('CAS-is-MeSH',len(cas2mesh), len(mesh2cas))

# Add items
key2mesh_via_db(cas2db, db2mesh, cas2mesh, mesh2cas)
print('CAS-is-MeSH',len(cas2mesh), len(mesh2cas))

# Export
json.dump(switch_dictset_to_dictlist(cas2mesh), open(direct+'cas2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2cas), open(direct+'mesh2cas.json','w'))

CAS-is-MeSH 6992 7012
CAS-is-MeSH 6992 7012


## ChEBI is MeSH

In [109]:
# Import
chebi2db = json.load(open(direct+'chebi2db.json'))

# Add items
chebi2mesh, mesh2chebi = dict(), dict()
print('ChEBI-is-MeSH', len(chebi2mesh))
key2mesh_via_db(chebi2db, db2mesh, chebi2mesh, mesh2chebi)
print('ChEBI-is-MeSH', len(chebi2mesh))

# Export
json.dump(switch_dictset_to_dictlist(chebi2mesh), open(direct+'chebi2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2chebi), open(direct+'mesh2chebi.json','w'))

ChEBI-is-MeSH 0
ChEBI-is-MeSH 4026


## KEGG Compound -is- MeSH

In [110]:
keggcompound2db = json.load(open(direct+'keggcompound2db.json'))
db2mesh = json.load(open('output/drug2drug/db2mesh.json'))
keggcompound2mesh, mesh2keggcompound = dict(), dict()

# KEGG Compound -is- MeSH
key2mesh_via_db(keggcompound2db, db2mesh, keggcompound2mesh, mesh2keggcompound)

# Export
json.dump(switch_dictset_to_dictlist(keggcompound2mesh), open(direct+'keggcompound2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2keggcompound), open(direct+'mesh2keggcompound.json','w'))
print(len(keggcompound2db), len(db2keggcompound))

2340 2340


## KEGG Drug -is- MeSH

In [5]:
keggdrug2db = json.load(open(direct+'keggdrug2db.json'))
db2keggdrug = json.load(open(direct+'db2keggdrug.json'))
db2mesh = json.load(open('output/drug2drug/db2mesh.json'))
keggdrug2mesh, mesh2keggdrug = dict(), dict()

# KEGG drug -is- MeSH
key2mesh_via_db(keggdrug2db, db2mesh, keggdrug2mesh, mesh2keggdrug)

# Export
#json.dump(switch_dictset_to_dictlist(keggdrug2mesh), open(direct+'keggdrug2mesh.json','w'))
#json.dump(switch_dictset_to_dictlist(mesh2keggdrug), open(direct+'mesh2keggdrug.json','w'))
print(len(keggdrug2db), len(db2keggdrug))

2172 2172


## DrugBank -is- MeSH

In [111]:
'''DrugBank-is-inchikey-is-MESH'''
inchikey2mesh = json.load(open(direct+'inchikey2mesh.json'))

for inchikey, meshes in inchikey2mesh.items():
    try:
        dbs = inchikey2db[inchikey]
        for db in dbs:
            for mesh in meshes:
                db2mesh.setdefault(db,set()).add(mesh)
                mesh2db.setdefault(mesh,set()).add(db)
    except:
        continue
              
print('MeSH-is-DrugBank:',len(mesh2db))
print('Drugbank-is-MeSH:',len(db2mesh))

MeSH-is-DrugBank: 7437
Drugbank-is-MeSH: 8046


In [113]:
'''DrugBank-is-smiles-is-MESH'''
smiles2mesh = json.load(open(direct+'smiles2mesh.json'))

for smiles, meshes in smiles2mesh.items():
    try:
        dbs = smiles2db[smiles]
        for db in dbs:
            for mesh in meshes:
                db2mesh.setdefault(db,set()).add(mesh)
                mesh2db.setdefault(mesh,set()).add(db)
    except:
        continue
              
print('MeSH-is-DrugBank:',len(mesh2db))
print('Drugbank-is-MeSH:',len(db2mesh))
json.dump(switch_dictset_to_dictlist(db2mesh), open(direct+'db2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2db), open(direct+'mesh2db.json','w'))

MeSH-is-DrugBank: 7437
Drugbank-is-MeSH: 8046


In [115]:
### Import
mesh2dbold = json.load(open('output/drug2drug/mesh2db.json'))
print('Old MeSH is DrugBank', len(mesh2dbold))
db2meshold = json.load(open('output/drug2drug/db2mesh.json'))
print('Old DrugBank is MeSH', len(db2meshold))
db2mesh = json.load(open(direct+'db2mesh.json'))
db2mesh = switch_dictlist_to_dictset(db2mesh)
db2meshold = json.load(open('output/drug2drug/db2mesh.json'))
db2meshold = switch_dictlist_to_dictset(db2meshold)

### Add items
for db in db2mesh.copy():
    if db in db2meshold:
        meshes = db2meshold[db]
        for mesh in meshes:
            db2mesh[db].add(mesh)
for db in db2meshold.copy():
    if db not in db2mesh:
        db2mesh[db] = db2meshold[db] 
        
json.dump(switch_dictset_to_dictlist(db2mesh), open(direct+'db2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2db), open(direct+'mesh2db.json','w'))
len(db2mesh)

Old DrugBank is MeSH 7939


8046

In [6]:
db2mesh

{'DB01509': ['D015104'],
 'DB06637': ['D015761'],
 'DB06262': ['D015103'],
 'DB01440': ['C111420'],
 'DB02362': ['D010129'],
 'DB00159': ['D015118'],
 'DB04699': ['D015107'],
 'DB02959': ['D006916'],
 'DB14010': ['D008732'],
 'DB01033': ['D015122'],
 'DB12231': ['D000002'],
 'DB00154': ['D015126'],
 'DB11145': ['D015125'],
 'DB00513': ['D015119'],
 'DB13262': ['C084650'],
 'DB01193': ['D000070'],
 'DB01418': ['D000074'],
 'DB00414': ['D000092'],
 'DB00316': ['C474747', 'D000082'],
 'DB03166': ['D019342'],
 'DB09347': ['D000100'],
 'DB00819': ['D000086'],
 'DB08842': ['D000108'],
 'DB03128': ['D000109'],
 'DB15906': ['D000114'],
 'DB06151': ['D000111'],
 'DB11617': ['D015250', 'C011157'],
 'DB15794': ['D000183'],
 'DB00141': ['D000117'],
 'DB12860': ['D000186'],
 'DB00919': ['D000198'],
 'DB09278': ['D002606'],
 'DB00173': ['D000225'],
 'DB00131': ['D000249'],
 'DB00061': ['D000243', 'C051409'],
 'DB00787': ['D000212'],
 'DB00640': ['D000241'],
 'DB15795': ['C084829'],
 'DB03708': ['D00

In [2]:
db2mesh = json.load(open('output/compound2compound/db2mesh.json'))

In [4]:
output_edgefile_onerel_noweight(outpath = 'output/compound2compound/edges_drugbank_is_mesh.csv',
                               columns = ['Compound (DrugBank)','Compound (MeSH)','Relationship'],
                               dictionary = db2mesh,
                               rel = '-is-',
                               prefix_col1 = 'DrugBank_Compound:',
                               prefix_col2 = 'MeSH_Compound:')

## CAS is MeSH

In [116]:
# Import
cas2mesh = switch_dictlist_to_dictset(cas2mesh)
print('CAS-is-MeSH',len(cas2mesh), len(mesh2cas))

# Add items
key2mesh_via_db(cas2db, db2mesh, cas2mesh, mesh2cas)
print('CAS-is-MeSH',len(cas2mesh), len(mesh2cas))

# Export
cas2mesh = switch_dictset_to_dictlist(cas2mesh)
mesh2cas = switch_dictset_to_dictlist(mesh2cas)
json.dump(cas2mesh, open(direct+'cas2mesh.json','w'))
json.dump(mesh2cas, open(direct+'mesh2cas.json','w'))

CAS-is-MeSH 6992 7012
CAS-is-MeSH 7778 7833


## ChEBI is MeSH

In [117]:
# Import
chebi2db = json.load(open(direct+'chebi2db.json'))

# Add items
print('ChEBI-is-MeSH', len(chebi2mesh))
key2mesh_via_db(chebi2db, db2mesh, chebi2mesh, mesh2chebi)
print('ChEBI-is-MeSH', len(chebi2mesh))

# Export
chebi2mesh = switch_dictset_to_dictlist(chebi2mesh)
mesh2chebi = switch_dictset_to_dictlist(mesh2chebi)
json.dump(chebi2mesh, open(direct+'chebi2mesh.json','w'))
json.dump(mesh2chebi, open(direct+'mesh2chebi.json','w'))

ChEBI-is-MeSH 4026
ChEBI-is-MeSH 4582


## KEGG Compound -is- MeSH

In [118]:
keggcompound2db = json.load(open(direct+'keggcompound2db.json'))
db2mesh = json.load(open('output/drug2drug/db2mesh.json'))

# KEGG Compound -is- MeSH
key2mesh_via_db(keggcompound2db, db2mesh, keggcompound2mesh, mesh2keggcompound)

# Export
json.dump(switch_dictset_to_dictlist(keggcompound2mesh), open(direct+'keggcompound2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2keggcompound), open(direct+'mesh2keggcompound.json','w'))
print(len(keggcompound2db), len(db2keggcompound))

2340 2340


## KEGG Drug -is- MeSH

In [6]:
keggdrug2db = json.load(open(direct+'keggdrug2db.json'))
db2keggdrug = json.load(open(direct+'db2keggdrug.json'))
db2mesh = json.load(open('output/drug2drug/db2mesh.json'))
keggdrug2mesh, mesh2keggdrug = dict(), dict()

# KEGG drug -is- MeSH
key2mesh_via_db(keggdrug2db, db2mesh, keggdrug2mesh, mesh2keggdrug)

# Export
json.dump(switch_dictset_to_dictlist(keggdrug2mesh), open(direct+'keggdrug2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2keggdrug), open(direct+'mesh2keggdrug.json','w'))
print(len(keggdrug2db), len(db2keggdrug))

2172 2172


## DrugBank -is- New TTD

In [3]:
''' Get DrugBank to Alt IDs: CAS, Chebi, PubChem Substance, PubChem Compound, oldTTD '''
db2oldttd = json.load(open('output/compound2compound/db2oldttd.json'))
db2chebi = json.load(open('output/compound2compound/db2chebi.json'))
db2pubchemC = json.load(open('output/compound2compound/db2pubchemcomp.json'))
db2pubchemS = json.load(open('output/compound2compound/db2pubchemsub.json'))
db2cas = json.load(open('output/compound2compound/db2cas.json'))
db2ChEMBL = json.load(open('output/compound2compound/db2chembl.json'))

print(len(db2oldttd), 'DrugBank to old TTD')
print(len(db2chebi), 'DrugBank to CheBI')
print(len(db2pubchemC), 'DrugBank to PubChem Compound')
print(len(db2pubchemS), 'DrugBank to PubChem Substance')
print(len(db2cas), 'DrugBank to CAS')
print(len(db2ChEMBL), 'DrugBank to ChEMBL')

1423 DrugBank to old TTD
5233 DrugBank to CheBI
8725 DrugBank to PubChem Compound
10452 DrugBank to PubChem Substance
9182 DrugBank to CAS
8020 DrugBank to ChEMBL


In [4]:
# Download TTD's External Mapping Drug file: http://db.idrblab.net/ttd/full-data-download
#! wget -P input/ http://db.idrblab.net/ttd/sites/default/files/ttd_database/P1-03-TTD_crossmatching.txt

In [5]:
''' Align New TTD Drug ID -is- xref Alt Drug ID '''
ttd2altids = dict(); info = []
with open('input/P1-03-TTD_crossmatching.txt','r') as fin:
    for count, line in enumerate(fin):
        if count < 28:
            continue
        if line == '\t\t\n':
            i = getTTD_IDs(info)
            ttd2altids[list(i.keys())[0]] = list(i.values())[0]
            info = []
            continue
        info.append(line.strip().split('\t'))
        
cas2ttd, pubchemc2ttd, pubchemsubs2ttd, chebi2ttd = dict(), dict(), dict(), dict()
align_ttd_to_xrefs(ttd2altids, cas2ttd, pubchemc2ttd, pubchemsubs2ttd, chebi2ttd)

            
''' Align New TTD -is- DrugBank '''
''' (NewTTD-is-AltID + AltID-is-DrugBank) '''
ttd2db = dict()
# Map DrugBank-PubChemC-TTD
key2value2ttd(db2pubchemC, pubchemc2ttd, ttd2db)

# Map DrugBank-PubChemSubs-TTD
key2value2ttd(db2pubchemS, pubchemsubs2ttd, ttd2db)

# Map DrugBank-CAS-TTD
key2value2ttd(db2cas, cas2ttd, ttd2db)
        
# Map DrugBank-Chebi-TTD
key2value2ttd(db2chebi, chebi2ttd, ttd2db)

In [6]:
''' Align OldTTD -is- NewTTD via DrugBank '''
#multiprocess_a_dict_of_lists_values(db2oldttd, getOld2NewTTDdrug)

# Merge temp files
oldttd2newttd = switch_dictset_to_dictlist(merge_oldttd2newttd())
json.dump(oldttd2newttd, open('output/compound2compound/oldTTD2newTTD.json','w'))
print(len(oldttd2newttd))

1291


In [7]:
''' Align DrugBank -is- New TTD '''
ttd2db = dict()
for db, oldttd in db2oldttd.items():
    try:
        newttd = oldttd2newttd[oldttd[0]][0]
        ttd2db.setdefault(newttd,set()).add(db)
    except:
        continue
ttd2db = switch_dictset_to_dictlist(ttd2db)
    

print('New TTD IDs mapping to multiple DrugBank IDs')
print_if_len_value_greater_than_1(ttd2db)

New TTD IDs mapping to multiple DrugBank IDs
D00YWP ['DB00984', 'DB08804']


In [8]:
json.dump(ttd2db, open('output/compound2compound/ttd2db.json','w'))

## KEGG

### Alignment: Compound to Drug
MeSH, KEGG Compound, KEGG Drug

In [4]:
keggcompound2mesh = json.load(open('output/compound2compound/keggcompound2mesh.json'))
mesh2keggcompound = json.load(open('output/compound2compound/mesh2keggcompound.json'))

keggdrug2mesh = json.load(open('output/compound2compound/keggdrug2mesh.json'))
mesh2keggdrug = json.load(open('output/compound2compound/mesh2keggdrug.json'))

keggdrug2db = json.load(open('output/compound2compound/keggdrug2db.json'))

keggcompound2db = json.load(open('output/compound2compound/keggcompound2db.json'))

In [5]:
''' KEGG Compund -is- KEGG Drug '''
keggcompound2keggdrug, keggdrug2keggcompound = dict(), dict()
for mesh, kegg_drugs in mesh2keggdrug.items():
    try:
        kegg_cpds = mesh2keggcompound[mesh]
        for kegg_cpd in kegg_cpds:
            for kegg_drug in kegg_drugs:
                keggcompound2keggdrug.setdefault(kegg_cpd,set()).add(kegg_drug)
                keggdrug2keggcompound.setdefault(kegg_drug,set()).add(kegg_cpd)
    except:
        continue

for mesh, kegg_compounds in mesh2keggcompound.items():
    try:
        kegg_drugs = mesh2keggdrug[mesh]
        
        for kegg_drug in kegg_drugs:
            for kegg_compound in kegg_compounds:
                keggdrug2keggcompound.setdefault(kegg_drug,set()).add(kegg_compound)
                keggcompound2keggdrug.setdefault(kegg_compound,set()).add(kegg_drug)
    except:
        continue
    
print(len(keggcompound2keggdrug), len(keggdrug2keggcompound), 'KEGG Compounds aligned to Drug / Compounds aligned to Drugs with MeSH alignments')

1063 1061 KEGG Compounds aligned to Drug / Compounds aligned to Drugs with MeSH alignments


In [8]:
for line in open('input/KEGG/kegg_drug_to_cpd.tsv'):
    line = line.split('\t')
    
    # KEGG Drug, KEGG Compund
    kegg_drug = line[0].split('dr:')[1]
    kegg_cpd = line[1].strip().split('cpd:')[1]
    
    # KEGG Drug -is- KEGG Compound
    keggcompound2keggdrug.setdefault(kegg_cpd,set()).add(kegg_drug)
    keggdrug2keggcompound.setdefault(kegg_drug,set()).add(kegg_cpd)
    
print(len(keggcompound2keggdrug), len(keggdrug2keggcompound), 'KEGG Compounds aligned to Drug / Compounds aligned to Drugs')

json.dump(switch_dictset_to_dictlist(keggcompound2keggdrug), open('output/compound2compound/keggcompound2keggdrug.json','w'))

1766 1775 KEGG Compounds aligned to Drug / Compounds aligned to Drugs
